In [1]:
import os

In [2]:
%pwd

'd:\\personal\\text-summarizer\\research'

In [12]:
os.chdir("../")

In [13]:
%pwd

'd:\\personal\\text-summarizer'

In [14]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_url:str
    local_data_file:Path
    unzip_dir:Path

In [15]:
Path().resolve().parent

WindowsPath('D:/personal')

In [16]:
os.getcwd()

'd:\\personal\\text-summarizer'

In [18]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories, get_size

class ConfigurationManager:
    def __init__(self, config_filepath:str=CONFIG_FILEPATH, params_filepath:str=PARAMS_FILEPATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(filepath=params_filepath)

        create_directories(path_to_directories=[self.config.artifacts_root])


    def get_data_ingestion_config(self,)->DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories(path_to_directories=[config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir
            , source_url=config.source_url
            , local_data_file=config.local_data_file
            , unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [19]:
# from textSummarizer.config import configuration

In [20]:
# from textSummarizer.config import configuration
from textSummarizer.logger import logger
import zipfile

In [21]:
import requests
from urllib import request

In [22]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config
    
    def download_file(self,):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url
                , filename = self.config.local_data_file
            )

            logger.info(f"{filename} downloaded..! with follwing info: \n{headers}")
        else:
            logger.info(f"file already exists..! Size: {get_size(Path(self.config.local_data_file))}")
        
    def extract_zip_file(self,):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            logger.info(f"Extracting files into {unzip_path}")
            zip_ref.extractall(unzip_path)

In [23]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-01-10 16:54:02,869: INFO: common: yaml file: D:\personal\text-summarizer\config\config.yaml loaded successfully..!]
[2025-01-10 16:54:02,869: INFO: common: yaml file: D:\personal\text-summarizer\params.yaml loaded successfully..!]
[2025-01-10 16:54:02,869: INFO: common: Created directory at: artifacts]
[2025-01-10 16:54:02,877: INFO: common: Created directory at: artifacts/data_ingestion]
[2025-01-10 16:54:04,440: INFO: 3305476219: artifacts/data_ingestion/data.zip downloaded..! with follwing info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: A954:57FCE:B9611:116964:6780F7F7
Accept-Ranges: bytes
Date: Fri, 10 Jan 2025 11:24:03 